In [1]:
import os
import pathlib

%cd /content

!pip install tf_slim lvis pycocotools tf-models-official

if not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models.git

/content
     |████████████████████████████████| 358kB 8.2MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 102kB 12.6MB/s 
     |████████████████████████████████| 37.6MB 85kB/s 
     |████████████████████████████████| 645kB 55.5MB/s 
     |████████████████████████████████| 174kB 64.0MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 1.2MB 48.9MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20072 sha256=7bd1c8495d8025ab213ee6eb0686d0443f1f5a55ebb1270c66ac216d569545c0
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=bf20341bb6075de63412843d7776f09302dfa505d536215d235eb330ca34f2e8
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built py

In [2]:
# To fix a depenency version crash in installing object detection
!pip install avro-python3==1.8.1

  Created wheel for avro-python3: filename=avro_python3-1.8.1-cp36-none-any.whl size=43164 sha256=a32c8ab0e5d3bf1595311204367d7a6e95dee09d98b5751395d26a01b33b385f
  Stored in directory: /root/.cache/pip/wheels/5c/04/3c/ffe3561c960133e747de503dea3e3facef2dea533bc92cb21a
Successfully built avro-python3


In [3]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp /content/models/research/object_detection/packages/tf2/setup.py /content/models/research
!python setup.py build
!python setup.py install

/content/models/research
running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/export_inference_graph.py -> build/lib/object_detection
copying object_detection/model_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/model_lib_v2.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/exporter_main_v2.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2.py -> build/lib/object_detection
copying object_detection/model_main_tf2

In [4]:
#Testing
!python object_detection/builders/model_builder_tf2_test.py

2021-01-24 04:32:19.955899: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-01-24 04:32:22.699272: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-24 04:32:22.700399: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-24 04:32:22.755839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-24 04:32:22.756401: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB devic

In [5]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [6]:
#Create working directory
%cd /content/
!mkdir workplace addon scripts
%cd workplace
!mkdir training_demo
%cd training_demo
!mkdir annotations exported-models images models pre_trained_models
%cd images
!mkdir train test
%cd /content/scripts
!mkdir preprocessing

/content
/content/workplace
/content/workplace/training_demo
/content/workplace/training_demo/images
/content/scripts


In [7]:
label_map_path="/content/workplace/training_demo/annotations/"
pretrain_model_dir=r"/content/workplace/training_demo/pre_trained_models"

In [8]:
#Generate label map
classes=['mouse']

def convert_classes(classes, start=1):
    msg = ''
    
    for id, name in enumerate(classes, start=start):
        msg = msg + "item {\n"
        msg = msg + " id: " + str(id) + "\n"
        msg = msg + " name: '" + name + "'\n}\n\n"
    return msg[:-1]

label_map = convert_classes(classes)

with open(label_map_path + "label_map.pbtxt", "w") as f:
    f.write(label_map)
    f.close()
print(f"label map with classes: {classes} is generated")

label map with classes: ['mouse'] is generated


In [9]:
#Download pre-trained model
def load_model(model_name):
  base_url = r'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
  model_file = model_name + '.tar.gz'
  pre_trained_file = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True,
    cache_subdir=pretrain_model_dir)

  return print('download pretrained model complete.')

In [10]:
model_name='ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
load_model(model_name)
%cd /content/workplace/training_demo/pre_trained_models
!rm *gz

244817920/244817203 [==============================] - 1s 0us/step
download pretrained model complete.
/content/workplace/training_demo/pre_trained_models


In [11]:
#Copy train and test data

%cd /content
!git clone https://github.com/Tungway1990/Tensorflow-object-detection-API.git

/content
Cloning into 'Tensorflow-object-detection-API'...
remote: Enumerating objects: 62, done.
remote: Total 62 (delta 0), reused 0 (delta 0), pack-reused 62
Unpacking objects: 100% (62/62), done.


In [12]:
%cd /content/Tensorflow-object-detection-API
!cp generate_tfrecord.py /content/scripts/preprocessing
!cp -r train /content/workplace/training_demo/images/
!cp -r test /content/workplace/training_demo/images/

/content/Tensorflow-object-detection-API


In [13]:
#xml to tfrecord
%cd /content/scripts/preprocessing
!python generate_tfrecord.py -x /content/workplace/training_demo/images/train -l /content/workplace/training_demo/annotations/label_map.pbtxt -o /content/workplace/training_demo/annotations/train.record
!python generate_tfrecord.py -x /content/workplace/training_demo/images/test -l /content/workplace/training_demo/annotations/label_map.pbtxt -o /content/workplace/training_demo/annotations/test.record

/content/scripts/preprocessing
Successfully created the TFRecord file: /content/workplace/training_demo/annotations/train.record
Successfully created the TFRecord file: /content/workplace/training_demo/annotations/test.record


In [14]:
%cd /content/workplace/training_demo/models
!mkdir my_ssd_resnet50_v1_fpn

/content/workplace/training_demo/models


In [15]:
%cd /content/workplace/training_demo/pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
!cp pipeline.config /content/workplace/training_demo/models/my_ssd_resnet50_v1_fpn/

/content/workplace/training_demo/pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8


In [16]:
#Update model pipeline config using regular expression
config_path=r'/content/workplace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config'
import re
'''Parameter Setting'''
num_classes=len(classes)
batch_size=8
fine_tune_checkpoint="/content/workplace/training_demo/pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0"
train_record_fname="/content/workplace/training_demo/annotations/train.record"
test_record_fname="/content/workplace/training_demo/annotations/test.record"
label_map_path=label_map_path+'label_map.pbtxt'

with open(config_path) as f:
    s = f.read()
with open(config_path, 'w') as f:
  # Set number of classes
  s = re.sub('num_classes: [0-9]+','num_classes: {}'.format(num_classes), s)
  # Set training batch_size.
  s = re.sub('batch_size: [0-9]+','batch_size: {}'.format(batch_size), s)
  # fine_tune_checkpoint
  s = re.sub('fine_tune_checkpoint: ".*?"','fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
  # fine_tune_checkpoint_type
  s = re.sub('fine_tune_checkpoint_type: ".*?"','fine_tune_checkpoint_type: "detection"', s)
  # label_map_path
  s = re.sub('label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_path), s) 
  # tfrecord files train and test.
  s = re.sub('input_path: "PATH_TO_BE_CONFIGURED"', 'input_path: "{}"'.format(train_record_fname),s, count=1)
  s = re.sub('input_path: "PATH_TO_BE_CONFIGURED"', 'input_path: "{}"'.format(test_record_fname), s)
  f.write(s)

In [17]:
!cp /content/models/research/object_detection/model_main_tf2.py /content/workplace/training_demo

In [18]:
%cd /content/workplace/training_demo

!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config --num_train_steps=15000

/content/workplace/training_demo
2021-01-24 04:33:46.756129: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-24 04:33:49.199814: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-24 04:33:49.200802: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-24 04:33:49.232309: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-24 04:33:49.232914: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2021-01-24 04:33:49.232957: I tensorflow/stream_ex

In [19]:
%cd /content/models/research/object_detection/
!cp /content/models/research/object_detection/exporter_main_v2.py /content/workplace/training_demo

%cd /content/workplace/training_demo/exported-models
!mkdir fine_tuned_model

%cd /content/workplace/training_demo
!python exporter_main_v2.py --input_type image_tensor \
--pipeline_config_path /content/workplace/training_demo/models/my_ssd_resnet50_v1_fpn/pipeline.config \
--trained_checkpoint_dir /content/workplace/training_demo/models/my_ssd_resnet50_v1_fpn \
--output_directory /content/workplace/training_demo/exported-models/fine_tuned_model

/content/models/research/object_detection
/content/workplace/training_demo/exported-models
/content/workplace/training_demo
2021-01-24 09:21:29.981230: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-24 09:21:34.884919: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-24 09:21:34.893986: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-24 09:21:34.930878: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-24 09:21:34.931429: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.

In [20]:
pb_fname = r'/content/workplace/training_demo/exported-models/fine_tuned_model/saved_model/saved_model.pb'
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [21]:
!ls -alh {pb_fname}

-rw-r--r-- 1 root root 12M Jan 24 09:22 /content/workplace/training_demo/exported-models/fine_tuned_model/saved_model/saved_model.pb


In [22]:
detection_model = tf.saved_model.load(r'/content/workplace/training_demo/exported-models/fine_tuned_model/saved_model')

In [23]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [24]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [25]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/workplace/training_demo/images/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('/content/workplace/training_demo/images/test/mice-and-cheese-h.jpg'),
 PosixPath('/content/workplace/training_demo/images/test/uXMPBS5sXRP_68FyqKwMUil7Pl6gT5F-WLZdAFi2XQA.jpg')]

In [26]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/workplace/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [27]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

Output hidden; open in https://colab.research.google.com to view.

In [28]:
#Save model
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r /content/workplace/training_demo/exported-models/fine_tuned_model/saved_model /content/gdrive/MyDrive

Mounted at /content/gdrive
